In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

In [ ]:
train_set = pd.read_csv("/kaggle/input/emnist/emnist-letters-train.csv")
test_set = pd.read_csv("/kaggle/input/emnist/emnist-letters-test.csv")

In [ ]:
train_set.head()

In [ ]:
train_set.info()

In [ ]:
print(f"Train set shape:  {train_set.shape}")
print(f"Test set shape:  {test_set.shape}")

In [ ]:
labels = train_set["23"].values
plt.figure(figsize=(20,6))
sns.countplot(x=labels)

In [ ]:
#train_set
y_train = np.array(train_set.iloc[:,0].values)
x_train = np.array(train_set.iloc[:,1:].values)
#test_set
y_test = np.array(test_set.iloc[:,0].values)
x_test = np.array(test_set.iloc[:,1:].values)
print(y_train.shape)
print(x_train.shape)

In [ ]:
fig, axes = plt.subplots(4, 5,figsize=(12,8))

for i, j in enumerate(axes.flat):
    j.imshow(x_train[i+2].reshape([28,28]))

In [ ]:
# Normalize pixel values to the range [0, 1] by dividing by 255.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape the data to have a single color channel (since EMNIST is grayscale)
# and match the input shape expected by the model
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

# One-hot encode the target labels for categorical classification.
y_train = tf.keras.utils.to_categorical(y_train, 37)  # 37 classes (26 letters + 1 for 'none')
y_test = tf.keras.utils.to_categorical(y_test, 37)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train ,test_size=0.2,random_state = 42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(512,(3,3), padding="same", activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(37,activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer="Adam",
             loss="categorical_crossentropy",
             metrics=(["accuracy"]))

In [ ]:
early_stopping_callback= EarlyStopping(monitor='val_accuracy',
                                       min_delta=0,
                                       verbose=0,
                                       restore_best_weights = True,
                                       patience=3,
                                       mode='max')

In [ ]:
epochs = 10
batch_size = 100

In [ ]:
history = model.fit(x_train, y_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test),
                   verbose=1,
                   callbacks=[early_stopping_callback])

if early_stopping_callback.stopped_epoch > 0:
    print(f"Training was stopped early at epoch {early_stopping_callback.stopped_epoch + 1} due to reaching the desired accuracy.")
else:
    print("Training completed without early stopping.")

In [ ]:
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()